# Smart Resource Selection and Orchestration

## Introduction

In this tutorial, we'll explore how OpenDXA agents can intelligently select and use different resources based on task requirements in both the planning and reasoning layers. Resource selection is a crucial capability for building versatile agents that can handle a variety of tasks efficiently.

You'll learn how to create custom resources, integrate Model Control Protocol (MCP) services, and use specialized agents as resources. We'll demonstrate how an agent automatically selects the most appropriate resource for each specific task, leading to more effective problem-solving.

### Learning Objectives

By the end of this tutorial, you will understand:

1. How resource selection works in the 2-layer architecture
2. How to create and register custom resources
3. How to use MCP services as resources
4. How to implement resource selection strategies
5. Best practices for resource orchestration

### Prerequisites

- Understanding of OpenDXA's 2-layer architecture
- Familiarity with resource management in OpenDXA
- Basic knowledge of MCP resources

## 1. Resource Selection in the 2-Layer Architecture

Resource selection happens in both layers:

- Planning layer: Selects resources for task planning
- Reasoning layer: Selects resources for task execution

Let's start with a basic example:

In [ ]:
from opendxa import ExecutionContext, Plan, PlanStep, ChainOfThoughtStrategy
from opendxa import BaseResource, ResourceSelector

# Create custom resources
class DataAnalysisResource(BaseResource):
    """Resource for data analysis tasks."""
    def __init__(self):
        super().__init__(
            name="data_analysis",
            description="Resource for data analysis tasks",
            capabilities=["analyze", "visualize"]
        )
    
    async def analyze(self, data):
        return {"result": "analysis complete"}
    
    async def visualize(self, data):
        return {"result": "visualization complete"}

class MLModelResource(BaseResource):
    """Resource for machine learning tasks."""
    def __init__(self):
        super().__init__(
            name="ml_model",
            description="Resource for machine learning tasks",
            capabilities=["predict", "train"]
        )
    
    async def predict(self, data):
        return {"result": "prediction complete"}
    
    async def train(self, data):
        return {"result": "training complete"}

# Create execution context
context = ExecutionContext()

# Register resources
context.register_resource(DataAnalysisResource())
context.register_resource(MLModelResource())

# Create resource selector
selector = ResourceSelector(context)

# Create a plan that uses resource selection
plan = Plan(
    name="resource_selection_plan",
    description="Plan demonstrating resource selection",
    objective="Execute tasks using appropriate resources"
)

# Add plan steps with resource selection
plan.add_step(PlanStep(
    name="analyze_data",
    description="Analyze data using appropriate resource",
    tool="analyze",
    tool_params={"data": [1, 2, 3, 4, 5]}
))

plan.add_step(PlanStep(
    name="predict_results",
    description="Predict results using appropriate resource",
    tool="predict",
    tool_params={"data": [1, 2, 3, 4, 5]}
))

# Create reasoning strategy
reasoning_strategy = ChainOfThoughtStrategy()

# Execute the plan
result = context.execute_plan(plan, reasoning_strategy)

print("Plan Execution Result:")
print(f"- Status: {result.status}")
print(f"- Completed Steps: {result.completed_steps}")
print(f"- Total Duration: {result.total_duration}")

## 2. Advanced Resource Selection Strategies

Let's explore more advanced resource selection strategies:

In [ ]:
from opendxa import ResourceSelectionStrategy

class CapabilityBasedStrategy(ResourceSelectionStrategy):
    """Strategy that selects resources based on capabilities."""
    
    def __init__(self, context):
        super().__init__(context)
        self._capability_scores = {
            "analyze": 1.0,
            "visualize": 0.8,
            "predict": 0.9,
            "train": 0.7
        }
    
    async def select_resource(self, task: dict) -> BaseResource:
        """Select the best resource for a given task."""
        required_capabilities = task.get("required_capabilities", [])
        
        # Score each resource based on capabilities
        resource_scores = {}
        for resource in self.context.resources:
            score = 0
            for capability in required_capabilities:
                if capability in resource.capabilities:
                    score += self._capability_scores.get(capability, 0.5)
            resource_scores[resource] = score
        
        # Select resource with highest score
        return max(resource_scores.items(), key=lambda x: x[1])[0]

class PerformanceBasedStrategy(ResourceSelectionStrategy):
    """Strategy that selects resources based on performance metrics."""
    
    def __init__(self, context):
        super().__init__(context)
        self._performance_metrics = {}
    
    async def select_resource(self, task: dict) -> BaseResource:
        """Select the best resource based on performance metrics."""
        # Update performance metrics
        self._update_metrics()
        
        # Select resource with best performance
        return max(
            self._performance_metrics.items(),
            key=lambda x: x[1]["score"]
        )[0]
    
    def _update_metrics(self):
        """Update performance metrics for resources."""
        for resource in self.context.resources:
            if resource not in self._performance_metrics:
                self._performance_metrics[resource] = {
                    "score": 0.5,
                    "success_rate": 1.0,
                    "response_time": 1.0
                }
            
            # Update metrics based on recent performance
            self._performance_metrics[resource]["score"] = \
                self._performance_metrics[resource]["success_rate"] * \
                (1.0 / self._performance_metrics[resource]["response_time"])

## 3. Resource Orchestration

Let's see how to orchestrate multiple resources:

In [ ]:
from opendxa import ResourceOrchestrator

class TaskOrchestrator(ResourceOrchestrator):
    """Orchestrator for managing multiple resources."""
    
    def __init__(self, context):
        super().__init__(context)
        self._resource_selector = CapabilityBasedStrategy(context)
    
    async def orchestrate_task(self, task: dict) -> dict:
        """Orchestrate resources for a given task."""
        # Select appropriate resource
        resource = await self._resource_selector.select_resource(task)
        
        # Execute task with selected resource
        result = await self._execute_with_resource(resource, task)
        
        # Handle any errors or retries
        if not result.get("success"):
            result = await self._handle_failure(resource, task, result)
        
        return result
    
    async def _execute_with_resource(self, resource: BaseResource, task: dict) -> dict:
        """Execute task with selected resource."""
        try:
            # Get appropriate method from resource
            method = getattr(resource, task["operation"])
            
            # Execute method with task parameters
            return await method(**task["parameters"])
        except Exception as e:
            return {"success": False, "error": str(e)}
    
    async def _handle_failure(self, resource: BaseResource, task: dict, result: dict) -> dict:
        """Handle task execution failure."""
        # Try alternative resource
        alternative_resource = await self._resource_selector.select_resource({
            **task,
            "exclude_resources": [resource]
        })
        
        if alternative_resource:
            return await self._execute_with_resource(alternative_resource, task)
        
        return result

## 4. Testing Resource Selection

Let's see how to test resource selection:

In [ ]:
import pytest

# Test data
TEST_TASK = {
    "operation": "analyze",
    "parameters": {"data": [1, 2, 3, 4, 5]},
    "required_capabilities": ["analyze"]
}

# Test resource selection
async def test_resource_selection():
    context = ExecutionContext()
    context.register_resource(DataAnalysisResource())
    context.register_resource(MLModelResource())
    
    selector = CapabilityBasedStrategy(context)
    resource = await selector.select_resource(TEST_TASK)
    
    assert resource.name == "data_analysis"
    assert "analyze" in resource.capabilities

# Test resource orchestration
async def test_resource_orchestration():
    context = ExecutionContext()
    context.register_resource(DataAnalysisResource())
    context.register_resource(MLModelResource())
    
    orchestrator = TaskOrchestrator(context)
    result = await orchestrator.orchestrate_task(TEST_TASK)
    
    assert result["success"]
    assert "result" in result

# Run the tests
if __name__ == "__main__":
    pytest.main([__file__])

## Next Steps

In this tutorial, we've covered:

1. Resource selection in the 2-layer architecture
2. Advanced resource selection strategies
3. Resource orchestration
4. Testing resource selection

In the next tutorial, we'll explore advanced resource management patterns and optimization techniques.